In [1]:
import os
from pathlib import Path
import concurrent.futures
from functools import partial
from tqdm import tqdm
import librosa
import soundfile as sf
from glob import glob

In [ ]:
audio_pathes = glob('/home/penguin/Data/cv-corpus-22.0-2025-06-20-ka/cv-corpus-22.0-2025-06-20/ka/clips/*.mp3')
new_root_path = '/home/penguin/Data/cv-corpus-22.0-2025-06-20-ka/cv-corpus-22.0-2025-06-20/ka/clips_16k/'

In [ ]:
def _resample_write(src_path: str, dst_dir: str) -> tuple[str, None | Exception]:
    try:
        sig, sr = librosa.load(src_path, sr=16000)
        dst = Path(dst_dir) / Path(src_path).name.replace('.mp3', '.flac')
        sf.write(str(dst), sig, 16000, format='FLAC')
        return (src_path, None)
    except Exception as e:
        return (src_path, e)

In [ ]:
# choose number of workers (tweak as needed)
max_workers = min(8, os.cpu_count() or 1)

In [ ]:
max_files = len(audio_pathes)
selected = audio_pathes[:max_files]

In [ ]:
errors = []
with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as ex:
    futures = {ex.submit(_resample_write, p, new_root_path): p for p in selected}
    for fut in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        src, err = fut.result()
        if err is not None:
            errors.append((src, err))

if errors:
    print(f"Completed with {len(errors)} errors. Example: {errors[:3]}")
else:
    print(f"Successfully processed {len(selected)} files.")

## Generat more audio times and text tokens for padding and static shapes

In [97]:
import pandas as pd
import string

In [98]:
root_path = '/home/penguin/Data/cv-corpus-22.0-2025-06-20-ka/cv-corpus-22.0-2025-06-20/ka'
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

In [99]:
original_df = pd.read_csv(Path(root_path) / 'train.tsv', delimiter='\t', usecols=['path', 'sentence'])

In [100]:
original_df['path'] = original_df['path'].str.replace('mp3', 'flac')
original_df['sentence'] = original_df['sentence'].str.translate(remove_punct_map).str.strip()

In [101]:
original_df['path'] = original_df['path'].apply(lambda x: Path(root_path) / 'clips_16k' / x)

In [102]:
def get_duration(path):
    sig, sr = librosa.load(Path(root_path) / 'clips_16k' / path, sr=None)
    return sig.shape[0] / sr

In [103]:
original_df['duration'] = original_df['path'].apply(get_duration)

In [104]:
from conformer.tokenizer import build_tokenizer
tokenizer = build_tokenizer(original_df['sentence'].values)

Vocabulary size: 41


In [105]:
original_df['label_token_count'] = original_df['sentence'].apply(lambda x: len(tokenizer.encode(x)))

In [106]:
original_df.to_csv(Path(root_path) / 'train_mod.tsv', sep='\t', index=False)

In [113]:
original_df

,path,sentence,duration,label_token_count
0,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,ქედის ფარგლებში არის დაცული ტერიტორიები,4.428,39
1,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,ტერიტორიის ნაწილს აკონტროლებს ჰარაკატ აშშაბააბ...,8.028,79
2,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,ადამიანები რომლებიც მედიტაციას მიმართავენ უფრო...,7.848,97
3,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,ადმინისტრაციული ცენტრია ქალაქი ონი,4.716,34
4,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,თუმცა შენობის ძირითადი ნაწილი მაშინ აშენდა და ...,7.920,82
...,...,...,...,...
62532,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,რაიონის მონადირეები ზრუნავდნენ სანადირო მეურნე...,5.328,68
62533,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,მის სიმღერებში შეიმჩნევა ჯაზური მუსიკის მკვეთრ...,4.536,55
62534,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,კინოკარიერის წლების განმავლობაში იგი სხვა შემო...,4.896,63
62535,/home/penguin/Data/cv-corpus-22.0-2025-06-20-k...,ჯეირახის რაიონი მდებარეობს ინგუშეთის მთიან ნაწ...,7.236,90


In [115]:
import jax

In [123]:
jax.devices('cpu')[0]

CpuDevice(id=0)

In [ ]:
int(4.428 * 16_000) - 1

70848

In [124]:
import jax.numpy as jnp

In [125]:
arr = jnp.array([1, 2, 3])

In [129]:
jnp.pad(arr, (0, 2), mode='constant', constant_values=0)

Array([1, 2, 3, 0, 0], dtype=int32)

In [112]:
original_df.iloc[0]['sentence']

'ქედის ფარგლებში არის დაცული ტერიტორიები'